# Alignment Error Visualization

This notebook collects COM data from the database and tries to quantify some alignment errors. The main results are shown in the plots at the end of the notebook.

In [1]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from collections import OrderedDict
from IPython.display import HTML
from itertools import combinations
from scipy.ndimage import affine_transform


PIPELINE_ROOT = Path('./').absolute().parents[1]
PIPELINE_ROOT = PIPELINE_ROOT.as_posix()
sys.path.append(PIPELINE_ROOT)
print(PIPELINE_ROOT)


/home/eddyod/programming/pipeline/src


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from library.controller.sql_controller import SqlController
from library.image_manipulation.filelocation_manager import FileLocationManager
from library.atlas.atlas_utilities import apply_affine_transform, get_affine_transformation, \
fetch_coms, list_coms, compute_affine_transformation, scale_coordinates
from library.atlas.brain_structure_manager import BrainStructureManager
from library.utilities.utilities_process import M_UM_SCALE, SCALING_FACTOR, random_string, \
read_image, write_image


XGBoost Version: 2.1.4


In [4]:
def sum_square_com(com):
    ss = np.sqrt(sum([s**2 for s in com]))
    return ss

def apply_affine_transformation(volume, matrix):
    """Apply an affine transformation to a 3D volume."""
    transformed_volume = affine_transform(volume, matrix, offset=0, order=1)
    return transformed_volume


In [103]:
moving_name = 'DK79'
fixed_name = 'MD589'

moving_all = list_coms(moving_name, scaling_factor=10)
fixed_all = list_coms(fixed_name, scaling_factor=10)

common_keys = list(moving_all.keys() & fixed_all.keys())
print(f'{moving_name} len={len(moving_all.keys())}')
print(f'{fixed_name} len={len(fixed_all.keys())}')

print(len(common_keys))

DK79 len=6
MD589 len=56
6


In [104]:
# these keys have very high errors: 'RtTg', 'AP'
midbrain_keys = {
            "3N_L",
            "3N_R",
            "4N_L",
            "4N_R",
            "IC",
            "PBG_L",
            "PBG_R",
            "SC",
            "SNC_L",
            "SNC_R",
            "SNR_L",
            "SNR_R",
        }
bad_keys = ('RtTg', 'AP', '3N_L', '3N_R')
bad_keys = ('RtTg', 'AP')
#bad_keys = ('RtTg',)
bad_keys = ()
good_keys = set(common_keys) - set(bad_keys)
print(f'#good_keys={len(good_keys)}')
print(good_keys)

#good_keys=6
{'coronal1', 'coronal2', 'Sagittal2', 'Sagittal3', 'coronal3', 'Sagittal1'}


In [105]:
moving_src = np.array([moving_all[s] for s in good_keys])
fixed_src = np.array([fixed_all[s] for s in good_keys])
transformation_matrix = compute_affine_transformation(moving_src, fixed_src)
print(repr(transformation_matrix))

array([[ 8.11052817e-01, -8.97891604e-02,  8.82937109e-02,
         9.66430191e+01],
       [ 3.70418947e-01,  6.27916376e-01, -1.59053155e-01,
        -1.19915965e+02],
       [ 4.63733778e-03,  8.57716398e-02,  8.37100186e-01,
         9.97117247e+01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])


In [84]:
# Apply affine transformation
# Apply affine transformation
origin = np.array([884, 697, 296])
print(origin)
trans_origin = apply_affine_transform(origin, transformation_matrix)
print(np.round(trans_origin))

[884 697 296]
[855. 734. 308.]


In [106]:
df_list = []
error = []
transformed_dict = {}
for structure in common_keys:
    moving0 = np.array(moving_all[structure])
    fixed0 = np.array(fixed_all[structure]) 
    transformed = apply_affine_transform(moving0, transformation_matrix)
    transformed = [x for x in transformed]
    difference = [a - b for a, b in zip(transformed, fixed0)]
    ss = sum_square_com(difference)
    row = [structure, np.round(moving0), np.round(fixed0), 
           np.round(transformed), np.round(difference), ss]
    df_list.append(row)
    error.append(ss)
    transformed_dict[structure] = transformed
print('RMS', sum(error)/len(df_list))
# MD589 to Allen RMS 260.0211852431133
# MD585 to Allen RMS 263.314352291951
# MD594 to Allen RMS 250.79820210419254
# AtlasV8 disk to Allen RMS 237.0680595008574
# MD585 to MD589 RMS 182.56958121184923

RMS 47.886577490886246


In [100]:
structure = 'VLL_L'
try:
    com = moving_all[structure]
except KeyError:
    structure = common_keys[0]
    com = moving_all[structure]
transformed_structure = apply_affine_transform(com, transformation_matrix)
print(f'{moving_name} {structure} {np.round(np.array(com))}')
print(f'{fixed_name} {structure} {np.round(np.array(fixed_all[structure]))}')
print(f'{moving_name} transformed {structure} {np.round(np.array(transformed_dict[structure]))}')
#print(f'neuro res {moving_name} {structure} {np.round(com)}')


DK79 coronal1 [947. 199. 409.]
MD589 coronal1 [824. 304. 429.]
DK79 transformed coronal1 [824. 304. 429.]


In [107]:
columns = ['structure', moving_name, fixed_name, 'transformed', 'difference', 'sumsquares']
df = pd.DataFrame(df_list, columns=columns)
df.index.name = 'Index'
df = df.round(4)
df.sort_values(by=['sumsquares'], inplace=True)
HTML(df.to_html(index=False))

structure,DK79,MD589,transformed,difference,sumsquares
coronal2,"[947.0, 349.0, 125.0]","[824.0, 437.0, 227.0]","[844.0, 430.0, 239.0]","[21.0, -7.0, 12.0]",24.7332
Sagittal3,"[1551.0, 756.0, 337.0]","[1311.0, 852.0, 447.0]","[1317.0, 876.0, 454.0]","[6.0, 24.0, 7.0]",25.3672
Sagittal1,"[167.0, 622.0, 337.0]","[230.0, 254.0, 447.0]","[206.0, 279.0, 436.0]","[-24.0, 25.0, -11.0]",36.3578
coronal3,"[947.0, 698.0, 349.0]","[824.0, 660.0, 457.0]","[833.0, 614.0, 456.0]","[9.0, -46.0, -1.0]",47.1088
coronal1,"[947.0, 199.0, 409.0]","[824.0, 304.0, 429.0]","[883.0, 291.0, 464.0]","[59.0, -13.0, 35.0]",69.7781
Sagittal2,"[1256.0, 211.0, 337.0]","[1197.0, 407.0, 447.0]","[1126.0, 424.0, 406.0]","[-71.0, 18.0, -41.0]",83.9743
